In [1]:
import os
import numpy as np
import random as rn
import tensorflow as tf

os.environ['PYTHONHASHSEED'] = '0'
np.random.seed(7)
rn.seed(7)

session_conf = tf.ConfigProto(
    intra_op_parallelism_threads=1,
    inter_op_parallelism_threads=1,
    gpu_options=tf.GPUOptions(
        visible_device_list="0", 
        allow_growth=True,
#         per_process_gpu_memory_fraction=0.5
    )
)

from keras import backend as K

tf.set_random_seed(7)
sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
K.set_session(sess)

import pickle
import pandas as pd

import xgboost as xgb
import lightgbm as lgb

from sklearn.ensemble import RandomForestClassifier as rf
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn import preprocessing
from keras import models
from keras import layers
from keras.layers.advanced_activations import LeakyReLU
from keras.utils import np_utils
from keras.layers import Dense, Dropout, Activation, Flatten, Input, Dense, GaussianNoise, GaussianDropout
from keras.models import Sequential, Model
from keras.regularizers import l2, l1
from keras.optimizers import Adam, SGD, Nadam
from keras.callbacks import LearningRateScheduler
from keras.metrics import categorical_accuracy
from keras.layers.normalization import BatchNormalization
from keras.callbacks import EarlyStopping
from keras.wrappers.scikit_learn import KerasClassifier
from  keras.regularizers import l1, l2
from keras import backend as K
from keras.utils.generic_utils import get_custom_objects

import matplotlib.pyplot as plt

%matplotlib inline

/home/yoshitaka-i/anaconda3/envs/py36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# read data
with open('../data/HCD35_pos.pickle', mode='rb') as fp:
    df_3 = pickle.load(fp)
    
df = pd.concat([
    df_3,
    pd.read_csv('../data/decomp_pos.csv').drop('Subclass', axis=1)
], axis=1)

In [3]:
# divide objective and target
objective = df.Subclass
le = preprocessing.LabelEncoder()
objective = le.fit_transform(objective)
features = df.drop('Subclass', axis=1)

# train test split
random_state=np.random.seed(42)
X_train, X_test, y_train, y_test = train_test_split(
    features, 
    objective,
    test_size=0.2
)

# transform  for keras's target label
y_train_for_keras = np_utils.to_categorical(y_train)
y_test_for_keras = np_utils.to_categorical(y_test)

In [4]:
tf.set_random_seed(42)

# make keras model

inputs = Input(shape=(X_train.shape[1],))

x = Dense(512, activation='relu')(inputs)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

predictions = Dense(len(df['Subclass'].value_counts()), activation='softmax')(x)
model = Model(inputs=inputs, outputs=predictions)

# compile
model.compile(
    loss='categorical_crossentropy',
    optimizer=Adam(),
    metrics=['accuracy']
)

# fit
epochs = 100
batch_size = 1000
es = EarlyStopping(monitor='val_loss', patience=20)

history = model.fit(
    X_train,
    y_train_for_keras,
    epochs=epochs,
    batch_size=batch_size,
    validation_data=(X_test, y_test_for_keras),
    verbose=1,
    callbacks=[
        es,
#         lr_decay
    ]
)

# model.save('../model/Keras_pos_HCD35+decomp.h5')

Train on 1340 samples, validate on 336 samples
Epoch 1/100
1340/1340 [==============================] - 7s 5ms/step - loss: 1.9351 - acc: 0.2619 - val_loss: 1.4507 - val_acc: 0.2738
Epoch 2/100
1340/1340 [==============================] - 3s 2ms/step - loss: 1.8850 - acc: 0.2530 - val_loss: 1.4266 - val_acc: 0.2827
Epoch 3/100
1340/1340 [==============================] - 3s 2ms/step - loss: 1.7269 - acc: 0.2873 - val_loss: 1.3415 - val_acc: 0.3333
Epoch 4/100
1340/1340 [==============================] - 3s 2ms/step - loss: 1.6690 - acc: 0.2821 - val_loss: 1.3188 - val_acc: 0.3482
Epoch 5/100
1340/1340 [==============================] - 3s 2ms/step - loss: 1.6483 - acc: 0.3000 - val_loss: 1.3053 - val_acc: 0.3720
Epoch 6/100
1340/1340 [==============================] - 3s 2ms/step - loss: 1.5493 - acc: 0.3291 - val_loss: 1.2874 - val_acc: 0.3839
Epoch 7/100
1340/1340 [==============================] - 3s 2ms/step - loss: 1.5109 - acc: 0.3313 - val_loss: 1.2795 - val_acc: 0.3661
Epoch 8/

Epoch 61/100
1340/1340 [==============================] - 3s 2ms/step - loss: 0.2797 - acc: 0.9075 - val_loss: 1.0480 - val_acc: 0.6696
Epoch 62/100
1340/1340 [==============================] - 2s 2ms/step - loss: 0.2613 - acc: 0.9142 - val_loss: 1.1036 - val_acc: 0.6637
Epoch 63/100
1340/1340 [==============================] - 2s 2ms/step - loss: 0.2494 - acc: 0.9157 - val_loss: 1.1803 - val_acc: 0.6429
Epoch 64/100
1340/1340 [==============================] - 3s 2ms/step - loss: 0.2362 - acc: 0.9172 - val_loss: 1.1701 - val_acc: 0.6488
Epoch 65/100
1340/1340 [==============================] - 2s 2ms/step - loss: 0.2158 - acc: 0.9224 - val_loss: 1.0441 - val_acc: 0.6875
Epoch 66/100
1340/1340 [==============================] - 2s 2ms/step - loss: 0.2412 - acc: 0.9254 - val_loss: 1.0971 - val_acc: 0.6935
Epoch 67/100
1340/1340 [==============================] - 2s 2ms/step - loss: 0.2513 - acc: 0.9164 - val_loss: 1.1461 - val_acc: 0.6696
Epoch 68/100
1340/1340 [========================

In [5]:
with open('../data/HCD35_neg.pickle', mode='rb') as fp:
    df_3 = pickle.load(fp)
    
df = pd.concat([
    df_3,
    pd.read_csv('../data/decomp_neg.csv').drop('Subclass', axis=1)
], axis=1)

In [6]:
# divide objective and target
objective = df.Subclass
le = preprocessing.LabelEncoder()
objective = le.fit_transform(objective)
features = df.drop('Subclass', axis=1)

# train test split
random_state=np.random.seed(42)
X_train, X_test, y_train, y_test = train_test_split(
    features, 
    objective,
    test_size=0.2
)

# transform  for keras's target label
y_train_for_keras = np_utils.to_categorical(y_train)
y_test_for_keras = np_utils.to_categorical(y_test)

In [7]:
tf.set_random_seed(42)

# make keras model

inputs = Input(shape=(X_train.shape[1],))

x = Dense(512, activation='relu')(inputs)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

predictions = Dense(len(df['Subclass'].value_counts()), activation='softmax')(x)
model = Model(inputs=inputs, outputs=predictions)

# compile
model.compile(
    loss='categorical_crossentropy',
    optimizer=Adam(),
    metrics=['accuracy']
)

# fit
epochs = 100
batch_size = 1000
es = EarlyStopping(monitor='val_loss', patience=20)

history = model.fit(
    X_train,
    y_train_for_keras,
    epochs=epochs,
    batch_size=batch_size,
    validation_data=(X_test, y_test_for_keras),
    verbose=1,
    callbacks=[
        es,
#         lr_decay
    ]
)

# model.save('../model/Keras_ng_HCD35+decomp.h5')

Train on 373 samples, validate on 94 samples
Epoch 1/100
373/373 [==============================] - 2s 7ms/step - loss: 2.0637 - acc: 0.2976 - val_loss: 1.2978 - val_acc: 0.4574
Epoch 2/100
373/373 [==============================] - 0s 302us/step - loss: 1.9129 - acc: 0.2922 - val_loss: 1.2515 - val_acc: 0.4681
Epoch 3/100
373/373 [==============================] - 0s 279us/step - loss: 1.9777 - acc: 0.3110 - val_loss: 1.2719 - val_acc: 0.4894
Epoch 4/100
373/373 [==============================] - 0s 253us/step - loss: 1.7449 - acc: 0.3271 - val_loss: 1.3240 - val_acc: 0.4681
Epoch 5/100
373/373 [==============================] - 0s 254us/step - loss: 1.6356 - acc: 0.3646 - val_loss: 1.3561 - val_acc: 0.4894
Epoch 6/100
373/373 [==============================] - 0s 247us/step - loss: 1.6897 - acc: 0.3995 - val_loss: 1.3509 - val_acc: 0.4894
Epoch 7/100
373/373 [==============================] - 0s 308us/step - loss: 1.6335 - acc: 0.3512 - val_loss: 1.3341 - val_acc: 0.5000
Epoch 8/100


Epoch 61/100
373/373 [==============================] - 0s 261us/step - loss: 0.6036 - acc: 0.7534 - val_loss: 1.0150 - val_acc: 0.6064
Epoch 62/100
373/373 [==============================] - 0s 311us/step - loss: 0.5858 - acc: 0.7507 - val_loss: 1.0183 - val_acc: 0.5957
Epoch 63/100
373/373 [==============================] - 0s 251us/step - loss: 0.5448 - acc: 0.7936 - val_loss: 1.0267 - val_acc: 0.6064
Epoch 64/100
373/373 [==============================] - 0s 310us/step - loss: 0.5846 - acc: 0.7507 - val_loss: 1.0342 - val_acc: 0.6170
Epoch 65/100
373/373 [==============================] - 0s 301us/step - loss: 0.5135 - acc: 0.7855 - val_loss: 1.0439 - val_acc: 0.6277
Epoch 66/100
373/373 [==============================] - 0s 251us/step - loss: 0.5184 - acc: 0.7936 - val_loss: 1.0503 - val_acc: 0.6383
